# Загрузка данных

Мы будем использовать [выборку Bank Marketing](https://quiltdata.com/package/uciml/bank).
Объектом в выборке является клиент банка, которому совершался звонок с предложением депозитного продукта.
Целевая переменная – согласие на продукт.

Для загрузки данных воспользуемся библиотекой [quilt](https://quiltdata.com).

```bash
pip install quilt
```

In [5]:
# Для первой загрузки
import quilt
quilt.install('uciml/bank')
from quilt.data.uciml import bank

bank_full = bank.tables.bank_full()
bank_full.head()

  0%|          | 0.00/12.8M [00:00<?, ?B/s]

100%|██████████| 12.8M/12.8M [00:01<00:00, 7.17MB/s]


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


# Подготовка данных

In [6]:
import numpy as np
import pandas as pd

features = ['age', 'job', 'marital', 'education', 'default', 'balance', 
            'housing', 'loan', 'contact', 'campaign',
            'pdays', 'previous', 'poutcome']  # 'duration'

cat_features = bank_full[features].select_dtypes(object).columns
num_features = bank_full[features].select_dtypes(np.number).columns

**Задание.** Произвести one hot кодировку категориальных полей.

Hint: [`pd.concat`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html), [`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html).

In [9]:
X = pd.concat([bank_full[num_features], pd.get_dummies(bank_full[cat_features])], axis=1)
X.shape

(45211, 37)

In [10]:
y = bank_full['y'].replace({'yes': 1, 'no': 0}) # ~ LabelEncoder()

Разделим выборку по времени

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Извлечение зависимости

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [13]:
cv = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

clf = GradientBoostingClassifier(learning_rate=0.1, n_estimators=10)

cv_score = cross_val_score(clf, X_train, y_train, cv=cv, scoring='roc_auc')
cv_score

array([0.71911506, 0.7578471 , 0.74462443])

---

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
booster_grid = {'learning_rate': [0.1, 0.05, 0.01], 'max_depth': [2, 3]}

booster = GradientBoostingClassifier(n_estimators=100)

gs_booster = GridSearchCV(booster, booster_grid, scoring='roc_auc',
                          cv=cv, verbose=1, n_jobs=-1, return_train_score=True)

In [16]:
gs_booster.fit(X_train, y_train);
gs_booster.best_params_

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   23.4s finished


{'learning_rate': 0.1, 'max_depth': 3}

In [17]:
# param_learning_rate - важно понимать, на что тратится больше времени
pd.DataFrame(gs_booster.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_learning_rate,param_max_depth,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,3.513916,0.042797,0.763322,0.773994,0.1,2,"{'learning_rate': 0.1, 'max_depth': 2}",2,0.742326,0.784598,0.779717,0.765063,0.767926,0.772321,0.029720,0.004291,0.015608,0.008062
1,5.293918,0.041146,0.765051,0.786433,0.1,3,"{'learning_rate': 0.1, 'max_depth': 3}",1,0.742973,0.796991,0.783234,0.778655,0.768949,0.783654,0.247224,0.000868,0.016666,0.007739
2,3.096929,0.037616,0.759354,0.766731,0.05,2,"{'learning_rate': 0.05, 'max_depth': 2}",4,0.738108,0.776826,0.777517,0.758625,0.762440,0.764743,0.048109,0.001032,0.016236,0.007563
3,5.541656,0.058827,0.762881,0.776501,0.05,3,"{'learning_rate': 0.05, 'max_depth': 3}",3,0.740280,0.786956,0.780231,0.768505,0.768135,0.774043,0.054786,0.007452,0.016728,0.007730
4,3.877026,0.071606,0.717524,0.718477,0.01,2,"{'learning_rate': 0.01, 'max_depth': 2}",6,0.698478,0.725478,0.738692,0.712773,0.715402,0.717180,1.026469,0.026604,0.016486,0.005267
5,7.188627,0.028485,0.741938,0.747013,0.01,3,"{'learning_rate': 0.01, 'max_depth': 3}",5,0.721123,0.757458,0.760244,0.736917,0.744451,0.746664,0.811240,0.005995,0.016070,0.008389


---

**Задание** Подберите параметр количества соседей для KNeighborsClassifier в диапозоне [1, 5, 10], делая при этом нормировку StandardScaler

Hint: для [`make_pipeline`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)

In [20]:
# для knn важно скалировать признаки (так чтобы стандартное отклонение равнялось 1 у всех признаков)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# чтобы указать, что параметр относится только к 1 шагу pipeline:
knn_grid = {'kneighborsclassifier__n_neighbors':[1,5,10]}

knn = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier()
)

gs_knn = GridSearchCV(knn, knn_grid, scoring='roc_auc', cv=cv, verbose=1, n_jobs=-1, return_train_score=True)

In [26]:
gs_knn.fit(X_train, y_train)
gs_knn.best_params_

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  2.2min finished


{'kneighborsclassifier__n_neighbors': 10}

In [27]:
pd.DataFrame(gs_knn.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_kneighborsclassifier__n_neighbors,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,1.753259,11.667274,0.589381,0.998707,1,{'kneighborsclassifier__n_neighbors': 1},3,0.582120,0.999076,0.593247,0.997844,0.592777,0.999202,0.574871,0.382580,0.005138,0.000613
1,1.079380,20.298389,0.669972,0.906407,5,{'kneighborsclassifier__n_neighbors': 5},2,0.660767,0.910913,0.671825,0.904915,0.677327,0.903392,0.530134,0.410243,0.006886,0.003246
2,0.804711,22.725230,0.697315,0.860777,10,{'kneighborsclassifier__n_neighbors': 10},1,0.679832,0.866974,0.708671,0.859664,0.703444,0.855692,0.217438,3.216899,0.012546,0.004673


0.998707 - качество почти 1: запомнили выборку и ближайший 1 объект к объекту - это он сам  
не 1 , потому что где-то два объекта с абсолютно одинаковыми признаками имеют разные классы

---

**Задание** Перед запуском Knn произведите отбор самых важных признаков, используя [`SelectFromModel`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html), перебирая значения `threshold` в диапозоне ['0.1*mean', '0.5*mean', 'mean'], а в качестве модели взять [`DecisionTreeClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

Применяем дерево решений и отбираем только такие фичи, у которых важность больше заданного значения.

Важно делать pipeline, потому что каждый раз нужно делать скалирование, отбор признаков и т п. Руками повторять не хочется.

In [43]:
# пример работы отбора признаков

In [42]:
tree = DecisionTreeClassifier(max_depth=10).fit(X_train, y_train)
pd.Series(tree.feature_importances_, index=X_train.columns).sort_values(ascending=False)

poutcome_success       0.400687
age                    0.157382
pdays                  0.105053
balance                0.078634
contact_unknown        0.052611
housing_no             0.036798
campaign               0.034978
previous               0.021562
loan_yes               0.009946
marital_single         0.009434
job_admin.             0.006739
job_blue-collar        0.006704
job_management         0.005842
education_primary      0.005840
job_technician         0.005781
education_tertiary     0.005557
education_secondary    0.005317
marital_married        0.005265
job_student            0.004804
contact_cellular       0.004780
loan_no                0.004675
marital_divorced       0.004560
job_retired            0.004367
education_unknown      0.004080
poutcome_failure       0.003538
job_unemployed         0.003452
job_entrepreneur       0.002266
contact_telephone      0.002142
job_self-employed      0.001365
poutcome_other         0.001357
job_services           0.001280
job_hous

In [ ]:
SelectFromModel() #shift enter - посмотреть подсказку

In [30]:
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier

SelectFromModel(DecisionTreeClassifier(max_depth=10)) - этому шагу возвращается обучающая выборка, на выходе набор   признаков. Каждому степу еще и целевая переменная подается.  
У дерева решения берется feature_importance и берется среднее по ним. Можно брать медиану или еще что.

In [48]:
knn_sel_grid = {'selectfrommodel__threshold':['0.1*mean', '0.5*mean', 'mean']}
knn_sel = make_pipeline(
    StandardScaler(),
    SelectFromModel(DecisionTreeClassifier(max_depth=10)),
    KNeighborsClassifier(n_neighbors=5)
)

gs_knn_sel = GridSearchCV(knn_sel, knn_sel_grid, scoring='roc_auc', cv=cv, verbose=1, n_jobs=-1, return_train_score=True)

In [49]:
gs_knn_sel.fit(X_train, y_train)
gs_knn_sel.best_params_

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   27.0s finished


{'selectfrommodel__threshold': '0.1*mean'}

Граница отбора получилась низкая => почти все признаки для knn важны.

In [50]:
pd.DataFrame(gs_knn_sel.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_selectfrommodel__threshold,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,1.684358,9.162720,0.673593,0.906391,0.1*mean,{'selectfrommodel__threshold': '0.1*mean'},1,0.663688,0.911274,0.679581,0.903425,0.677512,0.904475,0.554769,2.020069,0.007055,0.003479
1,0.882159,0.847610,0.655882,0.902966,0.5*mean,{'selectfrommodel__threshold': '0.5*mean'},3,0.645685,0.904813,0.657888,0.901566,0.664076,0.902519,0.348802,0.185500,0.007641,0.001363
2,0.336931,0.555693,0.657385,0.902291,mean,{'selectfrommodel__threshold': 'mean'},2,0.643997,0.908042,0.664443,0.899964,0.663717,0.898866,0.017227,0.091373,0.009472,0.004092


In [58]:
#информация о модели
knn_sel.set_params(selectfrommodel__threshold='0.1*mean')

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectfrommodel', SelectFromModel(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

# Stacking

In [62]:
booster = gs_booster.best_estimator_
knn = gs_knn.best_estimator_
knn_sel = gs_knn_sel.best_estimator_

**Задание** Написать функцию, которая возвращает прогноз по схеме кросс-валидации на обучающей выборке и на тесте.

Hint: не забудьте использовать метод `'predict_proba'` для [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)

Cross_val_predict 3 раза внутри обучает модель и делает предсказания на объектах каждого фолда, не используя информацию о них, и объединяет.

In [71]:
from sklearn.model_selection import cross_val_predict

def meta_feature(clf, X_train, y_train, X_test, cv):
    cv_pred_train = cross_val_predict(clf, X_train, y_train, method='predict_proba', cv=cv, n_jobs=-1)[:,1]
    clf.fit(X_train, y_train) # иначе clf получается обучен на последней выборке фолдов
    pred_test = clf.predict_proba(X_test)[:,1]
    return cv_pred_train, pred_test

Теперь используем все предсказанные ряды в качестве признаков.

In [74]:
meta_train = pd.DataFrame()
meta_test = pd.DataFrame()

meta_train['booster'], meta_test['booster'] = meta_feature(booster, X_train, y_train, X_test, cv)
meta_train['knn'    ], meta_test['knn'    ] = meta_feature(knn,     X_train, y_train, X_test, cv)
meta_train['knn_sel'], meta_test['knn_sel'] = meta_feature(knn_sel, X_train, y_train, X_test, cv)

In [75]:
from sklearn.metrics import roc_auc_score
meta_test.apply(lambda x: roc_auc_score(y_test, x))

booster    0.757439
knn        0.687338
knn_sel    0.660856
dtype: float64

In [76]:
meta_clf = GradientBoostingClassifier()
meta_clf.fit(meta_train, y_train)

roc_auc_score(y_test, meta_clf.predict_proba(meta_test)[:, 1])

0.7548658330286295

Стало хуже \\\_('')_/

In [78]:
!pwd #print working directory

/Users/a1/Documents/Летние школы/Тинькофф Финтех 2018/Семинары


In [79]:
meta_train.to_csv('meta_train.csv')
meta_test.to_csv('meta_test.csv')

---

## Домашнее Задание. Information value
Реализовать функцию подсчета [information value](http://ucanalytics.com/blogs/information-value-and-weight-of-evidencebanking-case). 

* в функции признак должен разбиваться на `num_buck` бакетов.
* для каждого бакета подсчитайте величину
$$
    \left(    \frac{tr}{all\_tr} -       \frac{1 - tr}{1 - all\_tr} \right) \cdot
    \left(log(\frac{tr}{all\_tr}) -  log(\frac{1 - tr}{1 - all\_tr})\right) \cdot
    \frac{cnt}{all\_cnt},
$$
    * $tr$ - доля целевой переменной в бакете (target rate)
    * $all\_tr$ - доля целевой переменной во всей выборке
    * $cnt$ - количество объектов в бакете
    * $all_cnt$ - количество обектов во всей выборке
    
* Если в бакете объекты только одного класса, то будут проблемы со взятием логарифма. Для этого значения $tr$ надо обрезать снизу – 0.001, а сверху 0.999 (hint: [`np.clip`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.clip.html))

In [ ]:
def information_value(feature, target, num_buck=10):
    # <вставить код>
    return iv

## Домашнее Задание. Важность признаков

Для некоторых задач важно уметь отобрать топ самых информативных признаков, выкинув неинформативный хвост.

Произвести сравнение различных способов подсчета важности признаков.

Способы подсчета важности:
* feature importance градиентного бустинга;
* information value;
* feature importance от random forest.

Способ сравнения:
* Фиксируйте одну модель по выбору.
* В качестве признаков нужно использовать топ-n от итоговых признаков по критерию важности.
* Построить зависимость качества на тесте от количества признаков n для каждого способа.

**Замечание**

Учесть, что первоначальному одному признаку соответствует несколько признаков после one hot кодирования, а в задании под принаком подразумеваются исходные (age, job, ...). При подсчете feature importance от категориальных полей нужно складывать feature importance всех получиных после one hot кодировки бинарных признаков.


Для каких способов совпадают наборы признаков для n = 1, 2, 3?  
Какой способ дает лучшее качество на тесте при использовании 5 признаков?

## Домашнее Задание. Stacking.

Попробовать стэкинг различных наборов моделей:
* Одну и туже модель с разными гиперпараметрами параметрами (например, бустинг с разным количеством деревьев);
* Разнообразные модели.

Попробовать различные мета классификаторы и простое усреднение.

Получилось ли за счет стэкинга улучшить качество?
Какое метод дает большее качество на тесте?